In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gradio as gr
import torch
from torchvision import transforms
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image
import cv2

In [ ]:
resnet18 = torchvision.models.resnet18()
#change output features from pretrain 1000 to 3 as we only have 3 classes
resnet18.fc = torch.nn.Linear(in_features=512, out_features=3)

resnet18.load_state_dict(torch.load('./Models/Resnet18-6/resnet18_epoch8.pth'))

<All keys matched successfully>

In [ ]:
#set learning rate as 3e-4
lr = 3e-4
#Declare criterion
loss_fn = torch.nn.CrossEntropyLoss()
#Declare adam optimizer with a learning rate of lr
optimizer = torch.optim.Adam(resnet18.parameters(), lr=lr)

In [ ]:
resnet18.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
labels = ['normal','thoraxdisease', 'covid']

In [ ]:
test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize(size=(224, 224)),
                                                 torchvision.transforms.ToTensor(),
                                                 torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
 

def test(inp):
  inp = Image.fromarray(inp, 'RGB')
  inp = transforms.Resize(size=(224, 224))(inp)
  inp = transforms.ToTensor()(inp).unsqueeze(0)
  inp = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(inp)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(resnet18(inp)[0], dim=0)
  return {labels[i]: float(prediction[i]) for i in range(3)}

In [ ]:
inputs = gr.inputs.Image()
outputs = gr.outputs.Label(num_top_classes=3)

examples=[["./Examples/covid1.png"],["./Examples/normal1.jpg"], ["./Examples/thorax1.jpeg"]]

io = gr.Interface(fn=test, inputs=inputs, outputs=outputs, examples=examples, title="Identifying COVID-19 Pneumonia", 
                  description="Predicts whether COVID-19 Penumonia is present in CXR. This model is EXPERIMENTAL and should only be used for research purposes. Please see a doctor for any diagnostic reasons. Please upload a Chest X-Ray image in JPG, JPEG  or PNG.").launch(debug=True)
io.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://13127.gradio.app
Interface loading below...


KeyboardInterrupt: ignored